# Olist CEO Dashboard Companion

This notebook mirrors the interactive Dash dashboard. Run it top to bottom to regenerate each figure for decks or executive briefings.



In [ ]:
from dashboard.data import load_dashboard_data
from dashboard.utils import METRIC_LABELS, format_brl, format_percent, format_category_name

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
data = load_dashboard_data()

print(f"Sections: {list(METRIC_LABELS.values())[:]}\n")
print("Financial overview keys:", data.financial_overview.keys())


## 1. Executive Overview

Waterfall chart mirroring the dashboard overview.



In [ ]:
data.waterfall_figure


In [ ]:
financials = data.financial_overview

kpi_summary = pd.DataFrame(
    {
        "Metric": [
            "Total revenue",
            "Net profit",
            "Net margin",
            "Active sellers",
        ],
        "Value": [
            format_brl(financials["revenues_total"]),
            format_brl(financials["profits_net"]),
            format_percent(financials["margin"]),
            f"{financials['seller_count']:,}",
        ],
    }
)

kpi_summary


## 2. Revenue Drivers

Monthly revenue trend and top categories (matching the dashboard selectors).



In [ ]:
monthly_metrics = data.monthly_metrics.copy()
selected_metrics = ["net_revenue", "olist_revenue", "reputation_cost"]
rename_map = {metric: METRIC_LABELS.get(metric, metric) for metric in selected_metrics}
trend_df = monthly_metrics[["month"] + selected_metrics].rename(columns=rename_map)

px.line(
    trend_df,
    x="month",
    y=list(rename_map.values()),
    markers=True,
    template="plotly_white",
    title="Monthly revenue vs. reputation drag",
)


In [ ]:
top_n = 10
category_df = data.category_profitability.nlargest(top_n, "net_profit").iloc[::-1].copy()
category_df["profit_margin"] = (
    category_df["net_profit"] / category_df["olist_commission"].replace(0, pd.NA)
).fillna(0.0)

px.bar(
    category_df,
    x="net_profit",
    y="product_category",
    orientation="h",
    text_auto=".2s",
    color="profit_margin",
    color_continuous_scale="Blues",
    hover_data={
        "net_profit": ":,.0f",
        "olist_commission": ":,.0f",
        "reputation_cost": ":,.0f",
        "order_count": True,
        "profit_margin": ":.1%",
    },
    template="plotly_white",
    title=f"Top {top_n} categories by net profit",
)


## 3. Customer Trust

Delivery delay vs. review score scatter plot with volume filter.



In [ ]:
state_df = data.state_metrics.copy()
min_orders = data.state_slider.default or 0
filtered_states = state_df[state_df["order_count"] >= min_orders].copy()

if filtered_states.empty:
    filtered_states = state_df.nlargest(10, "order_count").copy()

px.scatter(
    filtered_states,
    x="avg_delay",
    y="avg_review",
    size="olist_commission",
    color="avg_delay",
    hover_name="customer_state_name",
    hover_data={
        "avg_delay": ":.1f",
        "avg_review": ":.2f",
        "order_count": True,
        "olist_commission": ":,.0f",
    },
    template="plotly_white",
    color_continuous_scale="RdYlGn_r",
    title="Delivery delay vs. customer satisfaction",
).update_layout(yaxis=dict(range=[2.5, 5.1]))


## 4. Seller Strategy

What-if line chart for removing underperforming sellers.



In [ ]:
strategy_df = data.strategy_df.copy()
plot_df = strategy_df.melt(
    id_vars=["sellers_removed"],
    value_vars=["net_profit", "revenues", "total_costs"],
    var_name="Metric",
    value_name="BRL",
)

fig = px.line(
    plot_df,
    x="sellers_removed",
    y="BRL",
    color="Metric",
    template="plotly_white",
    markers=True,
    color_discrete_map={
        "net_profit": "#2563eb",
        "revenues": "#14b8a6",
        "total_costs": "#f97316",
    },
    title="Financial impact by sellers removed",
)
fig.add_vline(
    x=data.strategy_slider.default,
    line_dash="dash",
    line_color="#f97316",
    annotation_text="Default scenario",
    annotation_position="top right",
)
fig


## 5. CEO Next Moves

Recreate the numbers driving the recommendations panel.



In [ ]:
summary = pd.DataFrame(
    {
        "Headline": [
            "Latest monthly net revenue",
            "Profit uplift (profit-max scenario)",
            "Lean margin scenario",
        ],
        "Value": [
            format_brl(data.latest_net_revenue),
            format_brl(data.profit_uplift),
            format_percent(data.margin_strategy["margin"]),
        ],
        "Context": [
            f"Change vs. previous month: {format_brl(abs(data.net_revenue_change))}",
            f"Remove {int(data.profit_strategy['sellers_removed']):,} sellers",
            f"Baseline margin: {format_percent(data.baseline_strategy['margin'])}",
        ],
    }
)
summary


### Recommended actions

- Keep revenue momentum with the latest net revenue figure and CRM alignment.
- Stabilise delivery promise in the slowest state using playbooks from the fastest.
- Focus investment on hero categories driving net profit after reputation costs.
- Execute seller pruning plan to unlock profit uplift while maintaining partner base.
- Amplify promoters by replicating CX rituals from the top-rated state.

